In [4]:
import findspark
findspark.init('/home/pradyut/spark-3.2.3-bin-hadoop3.2-scala2.13')

In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Regression-Practise").getOrCreate()

In [6]:
df = spark.read.csv("/home/pradyut/Github_repos/PySpark-Essentials-Practise-main/Python_MLib/Ecommerce_Customers.csv",header=True,inferSchema=True)

# Linear Regression

In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

23/02/10 09:28:44 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [9]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [10]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')
lrmodel = lr.fit(training)

23/02/10 09:31:18 WARN Instrumentation: [358fbbf6] regParam is zero, which might cause numerical instability and overfitting.


In [11]:
lrmodel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [12]:
lrmodel.intercept

0.14228558260358093

In [13]:
training_summary = lrmodel.summary
print(training_summary.rootMeanSquaredError)

10.16309157133015


# Splitting Data and Evaluating based on test data

In [16]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")


23/02/10 09:38:47 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [17]:
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [18]:
train_data.count()

374

In [19]:
test_data.count()

127

In [20]:
correct_model = lr.fit(train_data)

23/02/10 09:40:15 WARN Instrumentation: [80c35f20] regParam is zero, which might cause numerical instability and overfitting.


In [27]:
correct_model_summary = correct_model.evaluate(test_data)

In [28]:
print(correct_model_summary)

In [29]:
correct_model_summary.meanSquaredError

114.72398563552126

In [30]:
correct_model_summary.devianceResiduals

[-23.194049550180548, 23.236905493283835]

In [32]:
correct_model_summary.residuals.show()

/home/pradyut/spark-3.2.3-bin-hadoop3.2-scala2.13/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------------+
|          residuals|
+-------------------+
| -19.27648804366447|
|-23.194049550180548|
|-19.700475003768638|
|-20.174103366907893|
|-19.486964299132243|
| -17.10764803802166|
|-16.838438339936115|
| -17.59797385451386|
|-16.500522044579938|
|-15.677949528598017|
|-15.019586090460805|
|-18.578355263137404|
| -14.29239302028616|
|-12.242460675603207|
|-14.593059027947369|
| -14.31059618260735|
|-10.113773708881233|
|-12.837533341384828|
|-11.194551463806027|
|-12.997494287815544|
+-------------------+
only showing top 20 rows



# Unlabelled Data

In [34]:
unlabelled = all_data.select("features")

In [36]:
unlabelled = unlabelled.limit(50)

In [37]:
unlabelled.count()

50

In [40]:
unlabelled.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [38]:
predictions = correct_model.transform(unlabelled)

In [39]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|   2.933660885400937|
|(10,[0,1,2,3,4,5,...|  0.8058122062089953|
|(10,[0,1,2,3,4,5,...|  0.7723330603662611|
|(10,[0,1,2,3,4,5,...|  0.5956519085939863|
|(10,[0,1,2,3,4,5,...|  1.3021777849692093|
|(10,[0,1,2,3,4,5,...| -2.9636678933420666|
|(10,[0,1,2,3,4,5,...|   1.338977602235281|
|(10,[0,1,2,3,4,5,...| 0.28983085751365584|
|(10,[0,1,2,3,4,5,...| -3.3520096625929097|
|(10,[0,1,2,3,4,5,...|   4.574795592359297|
|(10,[0,1,2,3,4,5,...|-0.06047229509786216|
|(10,[0,1,2,3,4,5,...|  2.0205506426538213|
|(10,[0,1,2,3,4,5,...|  3.2080102827949237|
|(10,[0,1,2,3,4,5,...| -0.3570076120205728|
|(10,[0,1,2,3,4,5,...|  0.4650521835382812|
|(10,[0,1,2,3,4,5,...| -2.6730742155480818|
|(10,[0,1,2,3,4,5,...|  -2.713856735959611|
|(10,[0,1,2,3,4,5,...| -2.3723916123336215|
|(10,[0,1,2,3,4,5,...|  3.2146760967124504|
|(10,[0,1,2,3,4,5,...|  0.789568